In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Charger les données
fichier = pd.read_csv("C:\\Users\\PC\\Desktop\\projet\\Web\\traitement_grav_final.csv")

# Préparer les données
X = fichier.drop(['Gravité', 'Disease'], axis=1)
y_disease = fichier['Disease']
y_gravity = fichier['Gravité']

# Standardisation des données explicatives
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train_disease, y_test_disease, y_train_gravity, y_test_gravity = train_test_split(
    X_scaled, y_disease, y_gravity, test_size=0.2, random_state=1)

# Paramètres pour la recherche par grille pour SVM
params_svm = {
    'C': [0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1],
    'kernel': ['rbf', 'poly']
}

# Modèle SVM pour prédire la présence d'une maladie
clf_disease_svm = GridSearchCV(SVC(), params_svm, cv=5, n_jobs=-1)
clf_disease_svm.fit(X_train, y_train_disease)
print("Meilleurs paramètres pour 'Disease':", clf_disease_svm.best_params_)
y_pred_disease = clf_disease_svm.predict(X_test)

# Affichage des résultats pour la prédiction de la maladie
print("Confusion Matrix for Disease Prediction:")
print(confusion_matrix(y_test_disease, y_pred_disease))
print("\nClassification Report for Disease Prediction:")
print(classification_report(y_test_disease, y_pred_disease))

# Filtrer les données d'entrainement pour la gravité
indices = (y_train_disease == 1)
X_train_gravity = X_train[indices]
y_train_gravity_filtered = y_train_gravity[indices]

# Modèle SVM pour prédire la gravité des crises
clf_gravity_svm = GridSearchCV(SVC(), params_svm, cv=5, n_jobs=-1)
clf_gravity_svm.fit(X_train_gravity, y_train_gravity_filtered)
print("Meilleurs paramètres pour 'Gravité':", clf_gravity_svm.best_params_)
indices_test = (y_pred_disease == 1)
y_pred_gravity = clf_gravity_svm.predict(X_test[indices_test])
actual_gravity = y_test_gravity[indices_test]

# Affichage des résultats pour la prédiction de la gravité
print("Confusion Matrix for Gravity Prediction among Predicted Cases:")
print(confusion_matrix(actual_gravity, y_pred_gravity))
print("\nClassification Report for Gravity Prediction among Predicted Cases:")
print(classification_report(actual_gravity, y_pred_gravity))


Meilleurs paramètres pour 'Disease': {'C': 0.1, 'gamma': 0.1, 'kernel': 'poly'}
Confusion Matrix for Disease Prediction:
[[6622    0]
 [   0  177]]

Classification Report for Disease Prediction:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6622
           1       1.00      1.00      1.00       177

    accuracy                           1.00      6799
   macro avg       1.00      1.00      1.00      6799
weighted avg       1.00      1.00      1.00      6799

Meilleurs paramètres pour 'Gravité': {'C': 1, 'gamma': 0.1, 'kernel': 'poly'}
Confusion Matrix for Gravity Prediction among Predicted Cases:
[[ 73   0]
 [  0 104]]

Classification Report for Gravity Prediction among Predicted Cases:
              precision    recall  f1-score   support

     moyenne       1.00      1.00      1.00        73
      élevée       1.00      1.00      1.00       104

    accuracy                           1.00       177
   macro avg       1.00    